## 1. 简介
> LruCache 算法就是 Least Recently Used，也就是最近最少使用算法。当缓存空间满了的时候，将最近最少使用的数据从缓存空间中删除以增加可用的缓存空间来缓存新内容。

---

## 2. 部分源码
### 2.1 成员变量及构造函数
- 内部使用 LinkedHashMap 作为缓存容器
- 构造函数中创建了 LinkedHashMap
  - 参数：容量、加载因子、排序模式
  - 排序模式: true 表示在访问的时候进行排序，否则只在插入的时候才排序
- 部分代码
```
public class LruCache<K, V> {
    // 真正放置缓存内容的 map
    private final LinkedHashMap<K, V> map;

    /** Size of this cache in units. Not necessarily the number of elements. */
    // 当前缓存已经使用用的大小，不一定是元素的个数
    private int size;
    private int maxSize;

    private int putCount;
    private int createCount;
    private int evictionCount;
    private int hitCount;
    private int missCount;

    /**
     * @param maxSize for caches that do not override {@link #sizeOf}, this is
     *     the maximum number of entries in the cache. For all other caches,
     *     this is the maximum sum of the sizes of the entries in this cache.
     */
    public LruCache(int maxSize) {
        if (maxSize <= 0) {
            throw new IllegalArgumentException("maxSize <= 0");
        }
        this.maxSize = maxSize;
        this.map = new LinkedHashMap<K, V>(0, 0.75f, true);
    }
    
    // 省略后续代码
    ......
}
```

---

### 2.2 get 方法
- 如果通过这个方法得到的需要的元素，那么这个元素会被放在缓存队列的头部

```
    /**
     * Returns the value for {@code key} if it exists in the cache or can be
     * created by {@code #create}. If a value was returned, it is moved to the
     * head of the queue. This returns null if a value is not cached and cannot
     * be created.
     */
    public final V get(K key) {
        if (key == null) {
            throw new NullPointerException("key == null");
        }

        V mapValue;
        // 这里用同步代码块
        synchronized (this) {
            // 从 LinkedHashMap 中获取数据
            mapValue = map.get(key);
            if (mapValue != null) {
                hitCount++;
                return mapValue;
            }
            missCount++;
        }

        /*
         * Attempt to create a value. This may take a long time, and the map
         * may be different when create() returns. If a conflicting value was
         * added to the map while create() was working, we leave that value in
         * the map and release the created value.
         */
        // 如果通过 key 从缓存集合中获取不到缓存数据，就尝试使用 creat(key)方法创造一个新数据。
        // create(key) 默认返回的也是 null，需要的时候可以重写这个方法。
        V createdValue = create(key);
        if (createdValue == null) {
            return null;
        }

        // 如果重写了 create(key) 方法，创建了新的数据，就将新数据放入缓存中。
        synchronized (this) {
            createCount++;
            
            // 如果 key 已经存在，会返回之前相同 key 的值
            mapValue = map.put(key, createdValue);

            // 如果之前存在相同 key 的 value，即有冲突。
            if (mapValue != null) {
                // There was a conflict so undo that last put
                // 有冲突，所以撤销刚才的操作，将之前相同 key 的值重新放回去
                map.put(key, mapValue);
            } else {
                size += safeSizeOf(key, createdValue);
            }
        }

        // 自定义方法，用于给使用者做一些统计等
        if (mapValue != null) {
            /*
             * 通过自定义方法 entryRemoved，通知使用者
             * 刚才 create 的值被删除了，之前相同 key 的值被重新添加回去了
             */
            entryRemoved(false, key, createdValue, mapValue);
            return mapValue;
        } else {
            trimToSize(maxSize);
            return createdValue;
        }
    }
```

---

### 2.3 put 方法
- 缓存中添加数据

```
    /**
     * Caches {@code value} for {@code key}. The value is moved to the head of
     * the queue.
     *
     * @return the previous value mapped by {@code key}.
     */
    public final V put(K key, V value) {
        if (key == null || value == null) {
            throw new NullPointerException("key == null || value == null");
        }

        V previous;
        synchronized (this) {
            putCount++;
            // safeSizeOf 方法返回的是 1，也就是将缓存的个数加 1。
            // 当缓存的是图片的时候，这个 size 应该表示图片占用的内存的大小
            // 所以应该重写里面调用的 sizeOf(key, value) 方法
            size += safeSizeOf(key, value);
            // 将创建的新元素添加进缓存队列，并在添加成功后返回这个元素
            previous = map.put(key, value);
            if (previous != null) {
                // 如果返回的是 null，说明添加缓存失败，在已用缓存大小中减去这个元素的大小。
                size -= safeSizeOf(key, previous);
            }
        }

        if (previous != null) {
            entryRemoved(false, key, previous, value);
        }

        trimToSize(maxSize);
        return previous;
    }
```

---

### 2.4 trimToSize 方法
- 移除最旧的缓存元素，直到剩余数量小于等于最大值

```
    /**
     * Remove the eldest entries until the total of remaining entries is at or
     * below the requested size.
     *
     * @param maxSize the maximum size of the cache before returning. May be -1
     *            to evict even 0-sized elements.
     */
    public void trimToSize(int maxSize) {
        // 开启一个死循环
        while (true) {
            K key;
            V value;
            synchronized (this) {
                if (size < 0 || (map.isEmpty() && size != 0)) {
                    throw new IllegalStateException(getClass().getName()
                            + ".sizeOf() is reporting inconsistent results!");
                }
                
                // 当已用的缓存小于最大缓存，完成任务，退出循环
                if (size <= maxSize) {
                    break;
                }

                // 在缓存队列中先找到最近最少使用的元素，
                // 调用 LinkedHashMap 的 eldest() 方法返回最不经常使用的方法。
                Map.Entry<K, V> toEvict = map.eldest();
                if (toEvict == null) {
                    break;
                }
                
                // 删掉这个元素，并减少已使用的缓存空间
                key = toEvict.getKey();
                value = toEvict.getValue();
                map.remove(key);
                size -= safeSizeOf(key, value);
                evictionCount++;
            }

            entryRemoved(true, key, value, null);
        }
    }
```

---

### 2.5 remove 方法
- 删除元素

```
    /**
     * Removes the entry for {@code key} if it exists.
     *
     * @return the previous value mapped by {@code key}.
     */
    public final V remove(K key) {
        if (key == null) {
            throw new NullPointerException("key == null");
        }

        V previous;
        synchronized (this) {
            // 直接移除
            previous = map.remove(key);
            if (previous != null) {
                // 减少已使用的缓存空间
                size -= safeSizeOf(key, previous);
            }
        }

        if (previous != null) {
            entryRemoved(false, key, previous, null);
        }

        return previous;
    }
```

---

### 2.6 entryRemoved 方法
- 覆写 entryRemoved 的作用
  - （put）put 发生 key 冲突时被调用，evicted=false，key=此次 put 的 key，oldValue=被覆盖的冲突 value，newValue=此次 put 的 value。
  - （trimToSize） trimToSize 的时候，只会被调用一次，就是最后一次被删除的最少访问数据带回来。evicted=true，key=最后一次被删除的 key，oldValue=最后一次被删除的 value，newValue=null（此次没有冲突，只是 remove）。
  - （remove） remove 的时候，存在对应 key，并且被成功删除后被调用。evicted=false，key=此次 put 的 key，oldValue=此次删除的 value，newValue=null（此次没有冲突，只是 remove）。
  - （get 后半段，查询丢失后处理情景，不过建议忽略） get 的时候，正常的话不实现自定义 create 的话，代码上看 get 方法只会走一半，如果你实现了自定义的 create(K key) 方法，并且在 create 后的值放入 LruCache 中发生 key 冲突时被调用，evicted=false，key=此次 get 的 key，oldValue=被你自定义 create(key) 后的 value，newValue=原本存在 map 里的 key-value。

```
    /**
     * 1. 当被回收或者删掉时调用。该方法当 value 被回收释放存储空间时被 remove 调用
     *    或者替换条目值时 put 调用，默认实现什么都没做。
     * 2. 该方法没用同步调用，如果其他线程访问缓存时，该方法也会执行。
     * 3. evicted=true：如果该条目被删除空间（表示进行了 trimToSize or remove）
     *    evicted=false：put 冲突后 或 get 里成功 create 后导致
     * 4. newValue != null，那么则被 put() 或 get() 调用。
     */
    protected void entryRemoved(boolean evicted, K key, V oldValue, V newValue) {}
```

### 2.7 create 方法
```

    /**
     * 一个空方法，也是在需要的时候重写实现
     */
    protected V create(K key) {
        return null;
    }
```

---

### 2.8 safeSizeOf 和 sizeOf 方法
- safeSizeOf：计算缓存占用大小，对于 item，按照个数计算；对于图片，按照图片占用内存大小计算
- sizeOf：默认返回 1，如果是图片 cache，需要重写此方法，返回图片的真实大小
- maxSize 和 sizeOf(K key, V value) 方法的覆写息息相关，必须相同单位。（比如 maxSize 是7MB，自定义的 sizeOf 计算每个数据大小的时候必须能算出与 MB 之间有联系的单位）

```
    private int safeSizeOf(K key, V value) {
        int result = sizeOf(key, value);
        if (result < 0) {
            throw new IllegalStateException("Negative size: " + key + "=" + value);
        }
        return result;
    }

    /**
     * Returns the size of the entry for {@code key} and {@code value} in
     * user-defined units.  The default implementation returns 1 so that size
     * is the number of entries and max size is the maximum number of entries.
     *
     * <p>An entry's size must not change while it is in the cache.
     */
    protected int sizeOf(K key, V value) {
        return 1;
    }
```

---

### 2.9 evictAll 方法
- 清空所有缓存

```
    /**
     * Clear the cache, calling {@link #entryRemoved} on each removed entry.
     */
    public final void evictAll() {
        trimToSize(-1); // -1 will evict 0-sized elements
    }
```

---

## 3. 为什么用 LinkedHashMap
### 3.1 构造函数
- LinkedHashMap 带有排序功能，3个参数的构造函数中，第三个参数表示在访问的时候是否需要排序，这个排序的结果就是把最近访问的数据放到集合的最后面。
- 这样，在删除的时候就从前面开始删除。
```
    /**
     * Constructs an empty <tt>LinkedHashMap</tt> instance with the
     * specified initial capacity, load factor and ordering mode.
     *
     * @param  initialCapacity the initial capacity
     * @param  loadFactor      the load factor
     * @param  accessOrder     the ordering mode - <tt>true</tt> for
     *         access-order, <tt>false</tt> for insertion-order
     * @throws IllegalArgumentException if the initial capacity is negative
     *         or the load factor is nonpositive
     */
    public LinkedHashMap(int initialCapacity,
                         float loadFactor,
                         boolean accessOrder) {
        super(initialCapacity, loadFactor);
        this.accessOrder = accessOrder;
    }
```

---

### 3.2 Entity的定义
- LinkedHashMap 内部是使用双向循环链表来存储数据的。
- 每一个元素都持有他上一个元素的地址和下一个元素的地址。

```
    /**
     * LinkedHashMap entry.
     */
    private static class LinkedHashMapEntry<K,V> extends HashMapEntry<K,V> {
        // These fields comprise the doubly linked list used for iteration.
        LinkedHashMapEntry<K,V> before, after;

        LinkedHashMapEntry(int hash, K key, V value, HashMapEntry<K,V> next) {
            super(hash, key, value, next);
        }

        /**
         * Removes this entry from the linked list.
         */
        private void remove() {
            before.after = after;
            after.before = before;
        }

        /**
         * Inserts this entry before the specified existing entry in the list.
         */
        // 将当前对象插入在 existingEntry 对象之前
        private void addBefore(LinkedHashMapEntry<K,V> existingEntry) {
            after = existingEntry;
            before = existingEntry.before;
            before.after = this;
            after.before = this;
        }

        /**
         * This method is invoked by the superclass whenever the value
         * of a pre-existing entry is read by Map.get or modified by Map.set.
         * If the enclosing Map is access-ordered, it moves the entry
         * to the end of the list; otherwise, it does nothing.
         */
        // 当集合的 get 方法被调用时，会调用这个方法。
        // 如果 accessOrder 为 true，就把这个元素放在集合的最末端。
        void recordAccess(HashMap<K,V> m) {
            LinkedHashMap<K,V> lm = (LinkedHashMap<K,V>)m;
            if (lm.accessOrder) {
                lm.modCount++;
                remove();
                addBefore(lm.header);
            }
        }

        void recordRemoval(HashMap<K,V> m) {
            remove();
        }
    }
```

---

### 3.3 get 方法的排序过程
- get 方法，支持重新排版

```
    /**
     * Returns the value to which the specified key is mapped,
     * or {@code null} if this map contains no mapping for the key.
     *
     * <p>More formally, if this map contains a mapping from a key
     * {@code k} to a value {@code v} such that {@code (key==null ? k==null :
     * key.equals(k))}, then this method returns {@code v}; otherwise
     * it returns {@code null}.  (There can be at most one such mapping.)
     *
     * <p>A return value of {@code null} does not <i>necessarily</i>
     * indicate that the map contains no mapping for the key; it's also
     * possible that the map explicitly maps the key to {@code null}.
     * The {@link #containsKey containsKey} operation may be used to
     * distinguish these two cases.
     */
    public V get(Object key) {
        LinkedHashMapEntry<K,V> e = (LinkedHashMapEntry<K,V>)getEntry(key);
        if (e == null)
            return null;
        e.recordAccess(this);
        return e.value;
    }
```

- 1. 初始化（before、after 都指向 head）
![image](get方法1.png)

- 2. 添加 A、B、C 3个数据
![image](get方法2.png)

- 3. 通过 get 方法访问数据 B
    - 先 remove
    - 再添加到 head 前面
![image](get方法3.png)
![image](get方法4.png)

---

### 3.4 清除最不常用的元素
- 获取使用最少的 item

```
    /**
     * Returns the eldest entry in the map, or {@code null} if the map is empty.
     *
     * Android-added.
     *
     * @hide
     */
    public Map.Entry<K, V> eldest() {
        Entry<K, V> eldest = header.after;
        return eldest != header ? eldest : null;
    }
```

---

## 4. 用 LruCache 来缓存 Bitmap 的初始化
- 初始化

```
LruCache<String, Bitmap> mLruCache;

// 获取手机最大内存(单位 kb)
int maxMemory = (int) (Runtime.getRuntime().maxMemory() / 1024);

// 一般都将 1/8 设为 LruCache 的最大缓存
int cacheSize = maxMemory / 8;

// 创建 LruCache
mLruCache = new LruCache<String, Bitmap>(maxMemory / 8) {
    /**
     * 重写这个方法，设置图片使用内存的真实大小
     * 缓存图片看的是占用的内存的大小，每张图片的占用内存也是不一样的，一次不能这样算。
     * 因此要重写这个方法，手动将这里改为本次缓存的图片的大小。
     */
    @Override
    protected int sizeOf(String key, Bitmap bitmap) {
        return bitmap.getByteCount() / 1024;
    }
};
```


- 使用

```
// 加入缓存
mLruCache.put("key", BitmapFactory.decodeResource(getResources(), R.mipmap.ic_launcher));

// 从缓存中读取
Bitmap bitmap = mLruCache.get("key");
```

---